In [1]:
# Imports
# source /Users/tnappy/node_projects/quickstart/python/bin/activate
# Read env vars from .env file
import base64
import os
import datetime as dt
import json
import time

from dotenv import load_dotenv
from flask import Flask, request, jsonify
import plaid
from plaid.model.payment_amount import PaymentAmount
from plaid.model.payment_amount_currency import PaymentAmountCurrency
from plaid.model.products import Products
from plaid.model.country_code import CountryCode
from plaid.model.recipient_bacs_nullable import RecipientBACSNullable
from plaid.model.payment_initiation_address import PaymentInitiationAddress
from plaid.model.payment_initiation_recipient_create_request import (
    PaymentInitiationRecipientCreateRequest,
)
from plaid.model.payment_initiation_payment_create_request import (
    PaymentInitiationPaymentCreateRequest,
)
from plaid.model.payment_initiation_payment_get_request import (
    PaymentInitiationPaymentGetRequest,
)
from plaid.model.link_token_create_request_payment_initiation import (
    LinkTokenCreateRequestPaymentInitiation,
)
from plaid.model.item_public_token_exchange_request import (
    ItemPublicTokenExchangeRequest,
)
from plaid.model.link_token_create_request import LinkTokenCreateRequest
from plaid.model.link_token_create_request_user import LinkTokenCreateRequestUser
from plaid.model.link_token_get_request import LinkTokenGetRequest
from plaid.model.asset_report_create_request import AssetReportCreateRequest
from plaid.model.asset_report_create_request_options import (
    AssetReportCreateRequestOptions,
)
from plaid.model.asset_report_user import AssetReportUser
from plaid.model.asset_report_get_request import AssetReportGetRequest
from plaid.model.asset_report_pdf_get_request import AssetReportPDFGetRequest
from plaid.model.auth_get_request import AuthGetRequest
from plaid.model.transactions_sync_request import TransactionsSyncRequest
from plaid.model.identity_get_request import IdentityGetRequest
from plaid.model.investments_transactions_get_request_options import (
    InvestmentsTransactionsGetRequestOptions,
)
from plaid.model.investments_transactions_get_request import (
    InvestmentsTransactionsGetRequest,
)
from plaid.model.accounts_balance_get_request import AccountsBalanceGetRequest
from plaid.model.accounts_get_request import AccountsGetRequest
from plaid.model.investments_holdings_get_request import InvestmentsHoldingsGetRequest
from plaid.model.item_get_request import ItemGetRequest
from plaid.model.institutions_get_by_id_request import InstitutionsGetByIdRequest
from plaid.model.transfer_authorization_create_request import (
    TransferAuthorizationCreateRequest,
)
from plaid.model.transfer_create_request import TransferCreateRequest
from plaid.model.transfer_get_request import TransferGetRequest
from plaid.model.transfer_network import TransferNetwork
from plaid.model.transfer_type import TransferType
from plaid.model.transfer_authorization_user_in_request import (
    TransferAuthorizationUserInRequest,
)
from plaid.model.ach_class import ACHClass
from plaid.model.transfer_create_idempotency_key import TransferCreateIdempotencyKey
from plaid.model.transfer_user_address_in_request import TransferUserAddressInRequest
from plaid.api import plaid_api

from decouple import config

load_dotenv()

True

In [2]:
# Fill in your Plaid API keys - https://dashboard.plaid.com/account/keys
PLAID_CLIENT_ID = config("PLAID_CLIENT_ID")
PLAID_SECRET = config("PLAID_SECRET")
# Use 'sandbox' to test with Plaid's Sandbox environment (username: user_good,
# password: pass_good)
# Use `development` to test with live users and credentials and `production`
# to go live
PLAID_ENV = config("PLAID_ENV", "sandbox")
# PLAID_PRODUCTS is a comma-separated list of products to use when initializing
# Link. Note that this list must contain 'assets' in order for the app to be
# able to create and retrieve asset reports.
PLAID_PRODUCTS = config("PLAID_PRODUCTS", "transactions").split(",")

# PLAID_COUNTRY_CODES is a comma-separated list of countries for which users
# will be able to select institutions from.
PLAID_COUNTRY_CODES = config("PLAID_COUNTRY_CODES", "US").split(",")


In [3]:
def empty_to_none(field):
    value = os.getenv(field)
    if value is None or len(value) == 0:
        return None
    return value

In [4]:
host = plaid.Environment.Sandbox

if PLAID_ENV == "sandbox":
    host = plaid.Environment.Sandbox

if PLAID_ENV == "development":
    host = plaid.Environment.Development

if PLAID_ENV == "production":
    host = plaid.Environment.Production

# Parameters used for the OAuth redirect Link flow.
#
# Set PLAID_REDIRECT_URI to 'http://localhost:3000/'
# The OAuth redirect flow requires an endpoint on the developer's website
# that the bank website should redirect to. You will need to configure
# this redirect URI for your client ID through the Plaid developer dashboard
# at https://dashboard.plaid.com/team/api.
PLAID_REDIRECT_URI = empty_to_none("PLAID_REDIRECT_URI")


In [5]:
configuration = plaid.Configuration(
    host=host,
    api_key={
        "clientId": PLAID_CLIENT_ID,
        "secret": PLAID_SECRET,
        "plaidVersion": "2020-09-14",
    },
)

api_client = plaid.ApiClient(configuration)
client = plaid_api.PlaidApi(api_client)

products = []
for product in PLAID_PRODUCTS:
    products.append(Products(product))

In [6]:
# We store the access_token in memory - in production, store it in a secure
# persistent data store.
access_token = None
# The payment_id is only relevant for the UK Payment Initiation product.
# We store the payment_id in memory - in production, store it in a secure
# persistent data store.
payment_id = None
# The transfer_id is only relevant for Transfer ACH product.
# We store the transfer_id in memory - in production, store it in a secure
# persistent data store.
transfer_id = None

item_id = None

In [7]:
# # https://plaid.com/docs/api/tokens/#linktokencreate
# request = LinkTokenCreateRequest(
#     products=products,
#     client_name="Plaid Quickstart",
#     country_codes=list(map(lambda x: CountryCode(x), PLAID_COUNTRY_CODES)),
#     language="en",
#     user=LinkTokenCreateRequestUser(client_user_id=str(time.time())),
# )
# if PLAID_REDIRECT_URI != None:
#     request["redirect_uri"] = PLAID_REDIRECT_URI


# # create link token
# response = client.link_token_create(request)
# link_token = response["link_token"]
# response

{'expiration': datetime.datetime(2023, 11, 3, 5, 32, 54, tzinfo=tzutc()),
 'link_token': 'link-sandbox-50f22052-10eb-448c-953d-27461310ace9',
 'request_id': 'dIG4QrmIPCxF5gX'}

In [8]:
# https://plaid.com/docs/api/tokens/#linktokenget
request = LinkTokenGetRequest(link_token=link_token)
response = client.link_token_get(request)
response

{'created_at': datetime.datetime(2023, 11, 3, 1, 32, 54, tzinfo=tzutc()),
 'expiration': datetime.datetime(2023, 11, 3, 5, 32, 54, tzinfo=tzutc()),
 'link_token': 'link-sandbox-50f22052-10eb-448c-953d-27461310ace9',
 'metadata': {'client_name': 'Plaid Quickstart',
              'country_codes': ['US', 'CA'],
              'initial_products': ['auth', 'transactions'],
              'language': 'en',
              'redirect_uri': None,
              'webhook': None},
 'request_id': 'KweH7EBI9Y82ET8'}

In [9]:
# https://plaid.com/docs/api/tokens/#itempublic_tokenexchange
public_token = link_token
request = ItemPublicTokenExchangeRequest(public_token=public_token)
response = client.item_public_token_exchange(request)
access_token = response['access_token']
item_id = response['item_id']

ApiException: Status Code: 400
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Server': 'nginx', 'Date': 'Fri, 03 Nov 2023 01:32:54 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '336', 'Connection': 'keep-alive', 'plaid-version': '2020-09-14'})
HTTP response body: {
  "display_message": null,
  "documentation_url": "https://plaid.com/docs/?ref=error#invalid-input-errors",
  "error_code": "INVALID_PUBLIC_TOKEN",
  "error_message": "provided token is the wrong type. expected \"public\", got \"link\"",
  "error_type": "INVALID_INPUT",
  "request_id": "EKZcEGav0jmt502",
  "suggested_action": null
}


In [10]:
# Get accounts from account_token
access_token = 'access-sandbox-b0297160-7715-41ca-abb8-44a8f5dc7d9e'
request = AccountsGetRequest(access_token=access_token)
response = client.accounts_get(request)
accounts = response['accounts']
response

{'accounts': [{'account_id': 'xvBmMrq6GBurq6pZMWrwSl5JZ6Rgbvc6jM5BA',
               'balances': {'available': 100.0,
                            'current': 110.0,
                            'iso_currency_code': 'USD',
                            'limit': None,
                            'unofficial_currency_code': None},
               'mask': '0000',
               'name': 'Plaid Checking',
               'official_name': 'Plaid Gold Standard 0% Interest Checking',
               'subtype': 'checking',
               'type': 'depository'},
              {'account_id': 'd1d9bwXRkdtA6yaV5PAjcKkDBob5E7CJKy8aG',
               'balances': {'available': 200.0,
                            'current': 210.0,
                            'iso_currency_code': 'USD',
                            'limit': None,
                            'unofficial_currency_code': None},
               'mask': '1111',
               'name': 'Plaid Saving',
               'official_name': 'Plaid Silver Standard

In [11]:
request = ItemGetRequest(access_token=access_token)
response = client.item_get(request)
item = response['item']
status = response['status']
response

{'item': {'available_products': ['assets',
                                 'balance',
                                 'signal',
                                 'identity',
                                 'investments',
                                 'liabilities'],
          'billed_products': ['auth', 'transactions'],
          'consent_expiration_time': None,
          'error': None,
          'institution_id': 'ins_56',
          'item_id': 'QlVAyJMRdVFKQj9XEmK8cRyrZamB4GcpAm1rB',
          'products': ['auth', 'transactions'],
          'update_type': 'background',
          'webhook': ''},
 'request_id': '7Krv7wxttccsFbc',
 'status': {'last_webhook': None,
            'transactions': {'last_failed_update': None,
                             'last_successful_update': datetime.datetime(2023, 11, 3, 1, 13, 42, 355000, tzinfo=tzutc())}}}

In [13]:
# This is a slower update
request = AccountsGetRequest(access_token=access_token)
response = client.accounts_get(request)
accounts = response['accounts']
response

{'accounts': [{'account_id': 'xvBmMrq6GBurq6pZMWrwSl5JZ6Rgbvc6jM5BA',
               'balances': {'available': 100.0,
                            'current': 110.0,
                            'iso_currency_code': 'USD',
                            'limit': None,
                            'unofficial_currency_code': None},
               'mask': '0000',
               'name': 'Plaid Checking',
               'official_name': 'Plaid Gold Standard 0% Interest Checking',
               'subtype': 'checking',
               'type': 'depository'},
              {'account_id': 'd1d9bwXRkdtA6yaV5PAjcKkDBob5E7CJKy8aG',
               'balances': {'available': 200.0,
                            'current': 210.0,
                            'iso_currency_code': 'USD',
                            'limit': None,
                            'unofficial_currency_code': None},
               'mask': '1111',
               'name': 'Plaid Saving',
               'official_name': 'Plaid Silver Standard